# Neural Style Transfer with VGG-19

In this project, we will implement a simple Neural Style Transfer algorithm. This algorithm was created by [Gatys et al. (2015).](https://arxiv.org/abs/1508.06576)

In [ ]:
#import libraries
import os
import sys
import scipy.io
import scipy.misc
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image
from nst_utils import *
import numpy as np
import tensorflow as tf
import pprint
%matplotlib inline

In [ ]:
#load parameters from VGG-19
pp = pprint.PrettyPrinter(indent=4)
model = load_vgg_model("pretrained-model/imagenet-vgg-verydeep-19.mat")
pp.pprint(model)

In [ ]:
#compute_content_cost

def compute_content_cost(a_C, a_G):
    
    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    a_C_unrolled = tf.reshape(a_C,[n_H*n_W,n_C])
    a_G_unrolled = tf.reshape(a_G,[n_H*n_W,n_C])
    
    J_content = (1/(4*n_H*n_W*n_C))*(tf.reduce_sum(tf.square(tf.subtract(a_C_unrolled,a_G_unrolled))))
    
    return J_content

In [ ]:
#define gram_matrix

def gram_matrix(A):
   
    GA = tf.matmul(A,A,transpose_b = True)
    
    return GA

In [ ]:
#compute cost of layer style

def compute_layer_style_cost(a_S, a_G):
   
    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    a_S = tf.transpose(tf.reshape(a_S,[n_H*n_W,n_C]))
    a_G = tf.transpose(tf.reshape(a_G,[n_H*n_W,n_C]))

    GS = gram_matrix(a_S)
    GG = gram_matrix(a_G)

    J_style_layer = (tf.reduce_sum(tf.square(tf.subtract(GS,GG))))/((2*n_C*n_H*n_W)**2)    
    
    return J_style_layer

In [ ]:
#define the weights of style_layers
STYLE_LAYERS = [
    ('conv1_1', 0.2),
    ('conv2_1', 0.2),
    ('conv3_1', 0.2),
    ('conv4_1', 0.2),
    ('conv5_1', 0.2)]

In [ ]:
def compute_style_cost(model, STYLE_LAYERS):

    J_style = 0

    for layer_name, coeff in STYLE_LAYERS:

        out = model[layer_name]
        a_S = sess.run(out)
        a_G = out
        
        J_style_layer = compute_layer_style_cost(a_S, a_G)
        J_style += coeff * J_style_layer

    return J_style

In [ ]:
# define the total cost

def total_cost(J_content, J_style, alpha = 10, beta = 40):

    J = alpha*J_content+beta*J_style
    
    return J

In [ ]:
#reset the graph
tf.reset_default_graph()

#start interactive session
sess = tf.InteractiveSession()

In [ ]:
#load the 'content' image
content_image = scipy.misc.imread("images/louvre_small.jpg")
content_image = reshape_and_normalize_image(content_image)

#load the 'style' image
style_image = scipy.misc.imread("images/monet.jpg")
style_image = reshape_and_normalize_image(style_image)

#generate noisy image
generated_image = generate_noise_image(content_image)
imshow(generated_image[0]);

#load pre-trained VGG-19 model
model = load_vgg_model("pretrained-model/imagenet-vgg-verydeep-19.mat")

In [ ]:
#assign the content image to be the input of the VGG model.  
sess.run(model['input'].assign(content_image))

#select the output tensor of layer conv4_2
out = model['conv4_2']

a_C = sess.run(out)
a_G = out

#compute the content cost
J_content = compute_content_cost(a_C, a_G)

#assign the input of the model to be the "style" image 
sess.run(model['input'].assign(style_image))

#compute the style cost
J_style = compute_style_cost(model, STYLE_LAYERS)

#compute total cost
J = total_cost(J_content, J_style, alpha = 10, beta = 40)

In [ ]:
# define optimizer
optimizer = tf.train.AdamOptimizer(2.0)

# define train_step
train_step = optimizer.minimize(J)

In [ ]:
def model_nn(sess, input_image, num_iterations = 200):

    sess.run(tf.global_variables_initializer())
  
    # Run the noisy input image (initial generated image) through the model. Use assign().
    sess.run(model['input'].assign(input_image))
    
    for i in range(num_iterations):
    
        generated_image = None

        if i%20 == 0:
            Jt, Jc, Js = sess.run([J, J_content, J_style])
            print("Iteration " + str(i) + " :")
            print("total cost = " + str(Jt))
            print("content cost = " + str(Jc))
            print("style cost = " + str(Js))
            
            save_image("output/" + str(i) + ".png", generated_image)
    
    save_image('output/generated_image.jpg', generated_image)
    
    return generated_image

In [ ]:
#generate new artistic
model_nn(sess, generated_image)